In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline


from abg_python.galaxy.gal_utils import Galaxy
from abg_python.all_utils import filterDictionary

from firestudio.studios.star_studio import StarStudio
from firestudio.studios.gas_studio import GasStudio
matplotlib.rcParams['figure.dpi'] = 150

In [ ]:
from firestudio.utils.stellar_utils import raytrace_projection,load_stellar_hsml

In [ ]:
import trove
import verdict

# Parameters

In [ ]:
pm = {
    'attenuate': False,
    'snum': 600,
#     'dynrange': 4089,
#     'dynrange': 3040, # m11q_md
    'dynrange': 12556,
#     'maxden': 7647435, # m11q_md
    'maxden': 4.99e8,
}

In [ ]:
pm = trove.link_params_to_config(
    '/home1/03057/zhafen/papers/Hot-Accretion-in-FIRE/analysis/hot_accretion.trove',
    variation = 'm12m',
    **pm
)

In [ ]:
# Halo file params
mt_kwargs = {
    'tag': 'smooth',
}

In [ ]:
store_child_ids = False

# Extract Galaxy

Create an abg_python$^\mathrm{TM}$ Galaxy instance

In [ ]:
snapdir = pm['sim_data_dir']
snapnum = pm['snum']
galaxy = Galaxy(
    'm12i_res7100',
    snapdir,
    snapnum,
    datadir = pm['data_dir'],
)

Locate the main halo, center our coordinates, and rotate to be face-on.

In [ ]:
galaxy.extractMainHalo()

In [ ]:
disk_mask = np.ones(galaxy.sub_star_snap['Masses'].size)
xs,ys,zs = galaxy.sub_star_snap['Coordinates'].T

disk_mask = (np.abs(xs) < 20) & (np.abs(ys) < 20) & (np.abs(zs) < 20)

In [ ]:
gas_disk_mask = np.ones(galaxy.sub_snap['Masses'].size)
gas_xs,gas_ys,gas_zs = galaxy.sub_snap['Coordinates'].T

gas_disk_mask = (np.abs(gas_xs) < 20) & (np.abs(gas_ys) < 20) & (np.abs(gas_zs) < 20)

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
segments = np.zeros((100,4))
segments[:,-1] = np.linspace(0,1,100)
gas_cmap = LinearSegmentedColormap.from_list('black_alpha_gradient',segments)

In [ ]:
from matplotlib.colors import LogNorm
h_m,xedges,yedges = np.histogram2d(
    xs[disk_mask],
    ys[disk_mask],
    bins=1200,
    weights=galaxy.sub_star_snap['Masses'][disk_mask]*1e10)
print(h_m.max())

dA = np.diff(xedges)[0]
dA*=dA

plt.imshow((h_m.T+1e-12)/dA,norm=LogNorm(1e6,1e9),origin='lower',cmap='afmhot')
plt.gca().set_aspect(1)

from matplotlib.colors import LogNorm
h_m,xedges,yedges = np.histogram2d(
    gas_xs[gas_disk_mask],
    gas_ys[gas_disk_mask],
    bins=1200,
    weights=galaxy.sub_snap['Masses'][gas_disk_mask]*1e10)
print(h_m.max())

dA = np.diff(xedges)[0]
dA*=dA

plt.imshow((h_m.T)/dA,norm=LogNorm(1e5,1e9),origin='lower',cmap=gas_cmap)
plt.gca().set_aspect(1)
#plt.colorbar(label='M')

In [ ]:
# Calculate smoothing length
galaxy.sub_star_snap['SmoothingLength'] = load_stellar_hsml.get_particle_hsml(
    galaxy.sub_star_snap['Coordinates'][:,0],
    galaxy.sub_star_snap['Coordinates'][:,1],
    galaxy.sub_star_snap['Coordinates'][:,2],
)

# StarStudio Usage

In [ ]:
# Star Studio is missing makeOutputDirectories
StarStudio.makeOutputDirectories = GasStudio.makeOutputDirectories
# In doing this the save directory gets messed up, but oh well.

In [ ]:
r_gal = 4. * galaxy.rstar_half

In [ ]:
if not pm['attenuate']:
    galaxy.sub_snap['Masses'] = np.zeros( galaxy.sub_snap['Masses'].shape )

In [ ]:
studio_datadir = os.path.join(os.path.dirname(galaxy.datadir),'firestudio')
print(studio_datadir)
my_starStudio = StarStudio(
    snapdir = snapdir,
    datadir = studio_datadir,
    snapnum = galaxy.snapnum,
    sim_name = pm['variation'],
    gas_snapdict=galaxy.sub_snap,
    star_snapdict=galaxy.sub_star_snap,
    frame_half_width =  2. * r_gal,
    frame_depth = 5. * r_gal,
    phi = 90,
    theta = 90,
    psi = 90,
    scale_bar = False,
    scale_bar_label = False,
)

In [ ]:
# FIRE studio will break if these aren't set.
my_starStudio.star_snapdict = galaxy.sub_star_snap
my_starStudio.gas_snapdict = galaxy.sub_snap
my_starStudio.sim_name = pm['variation']
my_starStudio.master_loud = True
my_starStudio.color_scheme_nasa = True
my_starStudio.Xmax = r_gal
my_starStudio.Xmin = -r_gal

In [ ]:
# Set automatically
my_starStudio.maxden = pm['maxden']
my_starStudio.dynrange = pm['dynrange']

In [ ]:
pixels = my_starStudio.render(plt.gca(),quick=False)
plt.gcf().set_dpi(120)
plt.gca().set_aspect( 'equal' )
plt.gcf()

## Save

### Save Figure

In [ ]:
# Save plot and metadata
image_name = 'edge_on'
my_starStudio.saveFigure( plt.gca(), image_name )

### Save Metadata

In [ ]:
image_name = "%s_%03d_%dkpc.hdf5" % (image_name,my_starStudio.snapnum, 2*my_starStudio.frame_half_width)
metadata_fp = os.path.join( my_starStudio.image_dir, image_name )
metadata = verdict.Dict.from_hdf5( metadata_fp, create_nonexistent=True )

In [ ]:
for attr_key in [ 'Xmin', 'Xmax', 'Ymin', 'Ymax', 'Zmin', 'Zmax', 'npix_x', 'npix_y', 'pixels', 'frame_half_width', 'frame_depth' ]:
    attr = getattr( my_starStudio, attr_key )
    metadata[attr_key] = attr

In [ ]:
metadata.to_hdf5( metadata_fp )
print( metadata_fp )

In [ ]:
plt.gcf()